In [ ]:
import alpaca_trade_api as alpaca
from alpaca.trading.client import TradingClient
from dotenv import load_dotenv
import os
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import datetime as dt
import statistics
from sklearn.linear_model import LinearRegression
import requests
import re

In [ ]:
paper = True

# initialize API from API keys in .env

load_dotenv('.env', override=True)

api_key = os.environ['APCA-API-PAPER-KEY-ID']
api_secret_key = os.environ['APCA-API-PAPER-SECRET-KEY']
api_base_url = 'https://paper-api.alpaca.markets/v2'

api = alpaca.REST(api_key, api_secret_key, api_base_url)
trading_client = TradingClient(api_key, api_secret_key, paper=paper)
account = trading_client.get_account()

In [ ]:
def get_beta(ticker, start_date='2020-01-01'):
    bars = api.get_bars(ticker, alpaca.TimeFrame.Day, start_date, adjustment='split').df
    QQQ_bars = api.get_bars("QQQ", alpaca.TimeFrame.Day, start_date, adjustment='split').df
    bars['change'] = bars['close'].diff()
    QQQ_bars['change'] = QQQ_bars['close'].diff()
    bars['percent change'] = bars['change'] / bars['close'] * 100
    QQQ_bars['percent change'] = QQQ_bars['change'] / QQQ_bars['close'] * 100
    bars = bars.dropna()
    QQQ_bars = QQQ_bars.dropna()

    
    # Assuming df1 and df2 are your dataframes and 'col1' and 'col2' are the columns
    x = QQQ_bars['percent change'].values.reshape(-1, 1)  # Independent variable (reshape for 2D array)
    y = bars['percent change'].values  # Dependent variable

    # Initialize the linear regression model
    model = LinearRegression()

    # Fit the model
    model.fit(x, y)

    # Get the slope (coefficient) and intercept of the regression line
    slope = model.coef_[0]
    return slope

def get_greeks(symbol):
    url = "https://data.alpaca.markets/v1beta1/options/snapshots?symbols=" + symbol + "&feed=indicative&limit=100"

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": api_secret_key
    }

    response = requests.get(url, headers=headers)
    data = response.json()
    # print(data)
    return data['snapshots'][symbol]['greeks']

def get_current_price(symbol):
    url = "https://data.alpaca.markets/v2/stocks/quotes/latest?symbols=" + symbol + "&feed=iex"

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": api_secret_key
    }

    response = requests.get(url, headers=headers)
    data = response.json()
    return (data['quotes'][symbol]['ap'] + data['quotes'][symbol]['bp']) / 2

def get_non_numeric_prefix(string):
    prefix = ""
    for char in string:
        if not char.isnumeric():
            prefix += char
        else:
            break
    return prefix

def get_QQQ_exposure():
    positions = trading_client.get_all_positions()
    QQQ_delta_exposure = 0
    QQQ_delta_hedge = 0
    QQQ_gamma_exposure = 0
    QQQ_gamma_hedge = 0

    QQQ_price = get_current_price('QQQ')
    for position in positions:
        symbol = position.symbol
        qty = position.qty
        ticker = get_non_numeric_prefix(symbol)
        if ticker == 'QQQ':
            greeks = get_greeks(symbol)
            delta = greeks['delta']
            gamma = greeks['gamma']
            QQQ_delta_hedge += float(qty) * delta
            QQQ_gamma_hedge += float(qty) * gamma
            continue
        price = get_current_price(ticker)
        greeks = get_greeks(symbol)
        delta = greeks['delta']
        gamma = greeks['gamma']
        beta = get_beta(ticker)
        QQQ_delta = float(qty) * delta * beta * price / QQQ_price 
        QQQ_delta_exposure += QQQ_delta
        QQQ_gamma = float(qty) * gamma * beta * beta * price * price / QQQ_price / QQQ_price
        QQQ_gamma_exposure += QQQ_gamma
    print(f"Total QQQ Delta Exposure: {QQQ_delta_exposure}")
    print(f"Total QQQ Delta Hedge: {QQQ_delta_hedge}")
    print()
    print(f"Total QQQ Gamma Exposure: {QQQ_gamma_exposure}")
    print(f"Total QQQ Gamma Hedge: {QQQ_gamma_hedge}")
    print()
    print(f"Combined QQQ Delta Exposure: {QQQ_delta_exposure + QQQ_delta_hedge}")
    print(f"Combined QQQ Gamma Exposure: {QQQ_gamma_exposure + QQQ_gamma_hedge}")
    print()
    print(f"2% (${"{:.2f}".format(QQQ_price * 0.02)}) move up: " + f"${"{:.2f}".format(QQQ_price * 2 * (QQQ_delta_exposure + QQQ_delta_hedge))} + " 
                        + f"${"{:.2f}".format(50 * ((QQQ_price * 0.02) ** 2) * (QQQ_gamma_exposure + QQQ_gamma_hedge))} = "
                        + f"${"{:.2f}".format(QQQ_price * 2 * (QQQ_delta_exposure + QQQ_delta_hedge) \
                            + 50 * ((QQQ_price * 0.02) ** 2) * (QQQ_gamma_exposure + QQQ_gamma_hedge))}")

    print(f"2% (${"{:.2f}".format(QQQ_price * 0.02)}) move down: " + f"${"{:.2f}".format(QQQ_price * -2 * (QQQ_delta_exposure + QQQ_delta_hedge))} + " 
                        + f"${"{:.2f}".format(50 * ((QQQ_price * 0.02) ** 2) * (QQQ_gamma_exposure + QQQ_gamma_hedge))} = "
                        + f"${"{:.2f}".format(QQQ_price * -2 * (QQQ_delta_exposure + QQQ_delta_hedge) \
                            + 50 * ((QQQ_price * 0.02) ** 2) * (QQQ_gamma_exposure + QQQ_gamma_hedge))}")
    print()

def get_equivalent_equity():
    positions = trading_client.get_all_positions()
    equities = {}
    for position in positions:
        symbol = position.symbol
        qty = position.qty
        ticker = get_non_numeric_prefix(symbol)
        price = get_current_price(ticker)
        greeks = get_greeks(symbol)
        delta = greeks['delta']
        equities[ticker] = equities.get(ticker, 0) + (float(qty) * price * 100 * delta)
    for ticker in equities:
        print(f"{ticker}: ${"{:.2f}".format(equities[ticker])}")
    print()

def get_net_theta():
    net_theta = 0
    positions = trading_client.get_all_positions()
    for position in positions:
        symbol = position.symbol
        qty = position.qty
        greeks = get_greeks(symbol)
        theta = greeks['theta']
        net_theta += float(qty) * theta * 100
    print(f"Net Theta: ${"{:.2f}".format(net_theta)}")
    print()

def get_options_df():
    positions = trading_client.get_all_positions()
    for position in positions:
        symbol = position.symbol
        print(symbol)
        qty = position.qty
        ticker = get_non_numeric_prefix(symbol)

def parse_option_symbol(symbol):
    # Regular expression to match the expected format
    pattern = r"^([A-Z.]+)(\d{6})([CP])(\d{8})$"
    match = re.match(pattern, symbol)
    
    if not match:
        raise ValueError("Invalid option symbol format")
    
    ticker, date, option_type, strike_price = match.groups()

    # Validate and parse expiration date
    try:
        expiration_date = datetime.strptime(date, "%y%m%d").strftime("%Y-%m-%d")
    except ValueError:
        raise ValueError("Invalid expiration date in option symbol")
    
    # Convert strike price (last 8 characters) to float with 3 decimal places
    strike_price = int(strike_price) / 1000
    
    # Return the parsed parts as a dictionary
    return {
        "ticker": ticker,
        "expiration_date": expiration_date,
        "option_type": "Call" if option_type == "C" else "Put",
        "strike_price": strike_price
    }

In [ ]:
get_QQQ_exposure()
get_equivalent_equity()
get_net_theta()

In [ ]:
get_options_df()